<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/091_Function_Calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



## **1. Why Function Calling Exists**

* **Old problem**: When you ask an LLM to output JSON, you often have to prompt-engineer it heavily — and even then it might return broken JSON, missing fields, or extra text.
* **Function calling API**: Lets you skip that dance. You give the model a **list of tools** (with JSON Schema for parameters), and it returns a *structured* `tool_call` object instead of messy free text.

---

## **2. How Function Calling Works**

1. **Define real Python functions**

   * These contain the actual business logic (e.g., `list_files`, `read_file`).
2. **Make a function registry**

   * A dictionary mapping function names → actual Python functions.
3. **Describe each tool in JSON Schema**

   * Includes `name`, `description`, `parameters` (with types & required fields).
4. **Send tools to the model in the API call**

   * `tools=[...]` parameter activates function calling mode.
5. **Model’s decision**

   * It can either return:

     * A structured `tool_call` (function name + JSON args)
     * Or just text (if no tool is needed)
6. **Run the called function**

   * You extract `tool_name` & `args` from `tool_calls[0]` and execute it.

---

## **3. What This Solves**

* **No more parsing headaches**: The API guarantees valid JSON for arguments.
* **Predictable structure**: Function names and parameters are always where you expect.
* **Mixed mode**: Model can choose between answering directly or calling a tool.
* **Cleaner prompts**: You don’t waste tokens on “always answer in this JSON format…”

---

## **4. Key Benefits for Agents**

* **Reliability** → Less code for parsing and fixing broken outputs.
* **Consistency** → Arguments match schema, no manual validation needed.
* **Maintainability** → Adding a new tool just means adding it to the tool list and registry.




In [1]:
!pip -q install openai python-dotenv

In [2]:
import os
from dotenv import load_dotenv
from openai import OpenAI

# --- 1. Load API key ---
load_dotenv('/content/API_KEYS.env')
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# --- 2. Define actual Python tool functions ---

def tool_list_files():
    import os
    return os.listdir('/content/files')

def tool_read_file(file_path: str):
    import os
    fp = os.path.join('/content/files', file_path)
    if not os.path.exists(fp):
        return f"File not found: {file_path}"
    with open(fp, 'r') as f:
        return f.read()

# --- 3. Tool registry ---
TOOLS_REGISTRY = {
    "list_files": tool_list_files,
    "read_file": tool_read_file,
}

# --- 4. Tool schemas for function calling ---
TOOLS_SCHEMAS = [
    {
        "type": "function",
        "function": {
            "name": "list_files",
            "description": "Returns a list of file names in /content/files",
            "parameters": {
                "type": "object",
                "properties": {}
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "read_file",
            "description": "Reads the content of a specified file in /content/files",
            "parameters": {
                "type": "object",
                "properties": {
                    "file_path": {"type": "string", "description": "The exact name of the file to read"}
                },
                "required": ["file_path"]
            }
        }
    }
]

# --- 5. Run one step of the agent ---

def run_agent(user_input: str):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are a file assistant. Use available tools if needed."},
            {"role": "user", "content": user_input},
        ],
        tools=TOOLS_SCHEMAS,
        tool_choice="auto"
    )

    msg = response.choices[0].message

    if msg.tool_calls:  # model decided to call a function
        for tool_call in msg.tool_calls:
            name = tool_call.function.name
            args = tool_call.function.arguments

            print(f"Model requested tool: {name}, args={args}")

            # Execute the matching Python function
            if name in TOOLS_REGISTRY:
                import json
                args_dict = json.loads(args) if args else {}
                result = TOOLS_REGISTRY[name](**args_dict)
                print("Tool result:", result)
            else:
                print(f"Unknown tool: {name}")
    else:
        print("Model replied with text:", msg.content)

# --- 6. Example run ---
run_agent("List all the files in /content/files")


Model requested tool: list_files, args={}
Tool result: ['003_gent Feedback and Memory.txt', '004_AGENT_Tools.txt', '002_Execute_the_Action.txt', '005_Using Function Calling Capabilities with LLMs.txt']




# What’s different now

* **No manual JSON parsing.** We stopped telling the model “please wrap JSON in `action`” and stopped scraping it with regex.

  * Before: `extract_markdown_block` → `json.loads` → handle errors.
  * Now: `msg.tool_calls` already has `name` and `arguments` as JSON.

* **Tools are passed via the API, not embedded in prose.**

  * Before: we stuffed tool specs (schemas/descriptions) into the system prompt.
  * Now: tool definitions go in the `tools=[...]` parameter. The model sees them in a structured, native way.

* **The model chooses `tool_choice="auto"`.**

  * Before: we forced it to output JSON every turn—even when no tool was needed.
  * Now: it can return **either** a function call **or** plain text.

# Why this improves the agent

* **Reliability up.** The API guarantees arguments are JSON; far fewer “broken JSON” or formatting failures.
* **Less prompt engineering.** You don’t need elaborate “always respond in JSON…” rules. Cleaner system prompts → fewer distractions.
* **Shorter code, simpler loop.** No parser, no error scaffolding around parsing. Easier to read and maintain.
* **Lower token usage (often).** You’re not reprinting big schemas in the prompt body each turn; the `tools` payload is handled separately by the API.
* **Mixed-mode behavior.** The model can just answer when a tool isn’t needed (e.g., “What is this directory for?”), and call tools when they are.
* **Easier to scale.** Adding a tool = add function + add entry in `tools` list + add to your registry. You don’t have to revise regexes or prompting tricks.

# Practical takeaways vs. previous code

1. **Parsing is gone.** Your agent logic becomes:
   `call model → if tool_calls: dispatch → else: print text`.
2. **Fewer failure modes.** Most “invalid JSON” cases disappear; remaining errors are actual runtime issues (missing file, wrong name), which are simpler to handle.
3. **Cleaner separation of concerns.**

   * Tool metadata lives in `tools=[...]`.
   * Tool implementation lives in Python.
   * The loop is tiny.
4. **Easier iteration.** You can prototype new tools fast, and you’re not constantly debugging formatting.

# Still keep in mind

* **Validate anyway.** Even with function calling, keep light checks (e.g., file exists, args present).
* **Multiple calls per turn.** The model can return more than one tool call; loop through `msg.tool_calls`.
* **Memory still matters.** You still decide what conversation/state the model sees each step.
* **Cost/latency.** Tools don’t fix those; smart memory policies still help.




